In [1]:
import tensorflow as tf
import utils.deeplabv3_ as build_model
import cv2
import numpy as np
import glob
import random
import utils.draw_predict as dp
import utils.call_data_v2 as cd

class_num = 2
learning_rate = 0.001
total_epoch = 200
batch_size = 1

img_path = '../dataset/trn_img/'
label_path = '../dataset/trn_labelmask/'

img_list = glob.glob(img_path + '*')
label_list = glob.glob(label_path + '*')

In [2]:
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, None, None, 3])
Y = tf.placeholder(tf.int32, shape=[None, None, None])  

test_img = cv2.imread('../dataset/trn_img/0_0.jpg')
test_img = test_img.reshape([1, test_img.shape[0], test_img.shape[1], test_img.shape[2]])

model = build_model.deeplabv3(X, class_num, True)
predict = tf.argmax(model, 3)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = model, labels = Y))
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_data = len(img_list)

random.seed(444)
random_indexes = random.sample(range(0, total_data), total_data)
total_batch = int(total_data / batch_size)

for i in range(total_epoch):
    print('Epoch : {} ['.format(i + 1), end='')

    total_cost = 0
    avg_cost = 0
    
    for batch_idx in range(total_batch):
        trn_x, trn_y = cd.read_batch(batch_idx, batch_size, img_list, label_list, random_indexes)
        
        _, c = sess.run([train_op, cost], feed_dict = {X : trn_x, Y : trn_y})
        
        total_cost += c
        
        if batch_idx % int(total_batch / 20) == 0:
            print('=', end='')
            
    avg_cost = total_cost / (total_batch + 1)

    print('] cost : {0:.7}'.format(avg_cost))

    p = sess.run([predict], feed_dict = {X : test_img})
    segmentation = dp.draw_pixel(p[0][0], class_num)
    cv2.imwrite('../dataset/trn_test/' + str(i) + '.png', segmentation)

    createFolder('../dataset/model/' + str(i))
    saver = tf.train.Saver()
    saver.save(sess, '../dataset/model/' + str(i) + '/' + str(i))

W1122 23:46:10.459161 31808 deprecation_wrapper.py:119] From C:\Users\th_k9\Desktop\pupil_segmentation\tensorflow_version\utils\deeplabv3_.py:154: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1122 23:46:10.462153 31808 deprecation_wrapper.py:119] From C:\Users\th_k9\Desktop\pupil_segmentation\tensorflow_version\utils\deeplabv3_.py:155: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W1122 23:46:10.463150 31808 deprecation.py:506] From C:\Users\th_k9\AppData\Local\Continuum\anaconda3\envs\Kimtae\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1122 23:46:10.485090 31808 deprecation_wrapper.py:119] From C:\Users\th_k9\Desktop\pupil_segmentatio

Epoch : 1 [=====================] cost : 0.1415537
Epoch : 2 [=====================] cost : 0.01251217
Epoch : 3 [=====================] cost : 0.004315174
Epoch : 4 [=====================] cost : 0.003304996
Epoch : 5 [=====================] cost : 0.003148833
Epoch : 6 [=====================] cost : 0.003105559
Epoch : 7 [=====================] cost : 0.003089407
Epoch : 8 [=====================] cost : 0.003076536
Epoch : 9 [=====================] cost : 0.003066645
Epoch : 10 [=====================] cost : 0.003056207
Epoch : 11 [=====================] cost : 0.003054712
Epoch : 12 [=====================] cost : 0.003049331
Epoch : 13 [=====================] cost : 0.003040515
Epoch : 14 [=====================] cost : 0.003035629
Epoch : 15 [=====================] cost : 0.003031173
Epoch : 16 [=====================] cost : 0.003026612
Epoch : 17 [=====================] cost : 0.003023161
Epoch : 18 [=====================] cost : 0.003019623
Epoch : 19 [=====================] cost 

In [ ]:
sess.close()

In [6]:
saver = tf.train.Saver()
saver.save(sess, '../dataset/model/total_model')

'../dataset/model/total_model'